<a href="https://colab.research.google.com/gist/theraw/c3b2ee748bf24a6295e50575f2bb34eb/copy-of-movie-downloader-gpu-encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 - Basic System update.

In [ ]:
!apt-get update
!apt-get upgrade -y
!apt-get dist-upgrade -y
!apt-get install build-essential libssl-dev curl nano wget zip unzip git lsb net-tools curl -y
!apt-get autoremove -y

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net

#2 - Install torrent client for downloading


In [ ]:
!add-apt-repository ppa:transmissionbt/ppa -y
!apt-get update -y
!apt-get upgrade -y
!apt-get install transmission-cli transmission-common transmission-daemon -y

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net

#3 - Configure Transmission

In [ ]:
!usermod -a -G debian-transmission root
!mkdir -p /movies/download
!mkdir -p /movies/encoded
!mkdir -p /movies/yml
!chgrp -R debian-transmission /movies/download
!chown -R debian-transmission:debian-transmission /movies/download
!service transmission-daemon start
!service transmission-daemon stop
!curl -s https://raw.githubusercontent.com/theraw/gcolab-encoder/main/static/settings.json > /etc/transmission-daemon/settings.json
!curl -s https://raw.githubusercontent.com/theraw/gcolab-encoder/main/static/exec_ngrok.sh > /movies/yml/ngrok.sh
!curl -s https://raw.githubusercontent.com/theraw/gcolab-encoder/main/static/services.sh > /movies/yml/services.sh
!chmod +x /movies/yml/services.sh
!chmod +x /movies/yml/ngrok.sh
!curl -s https://raw.githubusercontent.com/theraw/gcolab-encoder/main/static/tx.yml > /movies/yml/config.yml
!service transmission-daemon start


 * Starting bittorrent daemon transmission-daemon
/usr/bin/transmission-daemon already running.
   ...fail!
 * Stopping bittorrent daemon transmission-daemon
   ...done.
 * Starting bittorrent daemon transmission-daemon
   ...done.


#4 - Publish Transmission web gui with ngrok **(REQUIRED NGROK KEY)**

In [ ]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

/drive/ngrok-ssh
--2020-12-29 22:21:15--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.213.190.117, 54.225.42.45, 107.21.11.91, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.213.190.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  54.8MB/s    in 0.2s    

2020-12-29 22:21:16 (54.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip


In [ ]:
with open('/movies/yml/config.yml', 'r') as file :
  filedata = file.read()
filedata = filedata.replace('RPLXXXXXX', 'REQUIRED NGROK KEY HERE')
with open('/movies/yml/config.yml', 'w') as file:
  file.write(filedata)

In [ ]:
import os
get_ipython().system_raw('bash /movies/yml/ngrok.sh torrent web vcode &')

#5 Install Vcode just in case managing container from here looks hard..

In [ ]:
!curl -fsSL https://code-server.dev/install.sh | sh > /dev/null

#6 - Install nginx


In [ ]:
!apt-get install g++ flex bison curl doxygen libyajl-dev libgeoip-dev libtool dh-autoreconf libcurl4-gnutls-dev libxml2 libpcre++-dev libxml2-dev -y
!apt-get install libuuid1 uuid-dev -y
!apt-get install libgd-dev libc6 -y
!apt-get install libxslt-dev -y
!apt-get install libgd2-xpm -y
!apt-get install libgd2-xpm-dev -y
!apt-get install libgeoip-dev -y
!apt-get install libssl libssl-dev -y
!apt-get install checkinstall libpcre3 libpcre3-dev zlib1g zlib1g-dbg libxml2 zlib1g-dev -y
!cd /tmp/; wget http://nginx.org/download/nginx-1.19.6.tar.gz; tar xf nginx-1.19.6.tar.gz; rm -Rf nginx-1.19.6.tar.gz; mv nginx-1.19.6 nginx
!curl -s https://raw.githubusercontent.com/theraw/gcolab-encoder/main/static/cnginx.sh > /tmp/nginx/nbuild.sh
!cd /tmp/nginx && bash nbuild.sh && make -j`nproc` && make install
!rm -Rf /tmp/nginx
!useradd nginx
!curl -s https://raw.githubusercontent.com/theraw/gcolab-encoder/main/static/nginx.conf > /nginx/nginx.conf
!mkdir /movies/public_html
!chown -R nginx:nginx /movies/public_html

Reading package lists... Done
Building dependency tree       
Reading state information... Done
bison is already the newest version (2:3.0.4.dfsg-1build1).
dh-autoreconf is already the newest version (17).
doxygen is already the newest version (1.8.13-10).
flex is already the newest version (2.6.4-6).
libgeoip-dev is already the newest version (1.6.12-1).
libtool is already the newest version (2.4.6-2).
libyajl-dev is already the newest version (2.1.0-2build1).
libpcre++-dev is already the newest version (0.9.5-6.1).
curl is already the newest version (7.58.0-2ubuntu3.12).
g++ is already the newest version (4:7.4.0-1ubuntu2.3).
libcurl4-gnutls-dev is already the newest version (7.58.0-2ubuntu3.12).
libxml2 is already the newest version (2.9.4+dfsg1-6.1ubuntu1.3).
libxml2-dev is already the newest version (2.9.4+dfsg1-6.1ubuntu1.3).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Do

In [ ]:
import os
get_ipython().system_raw('bash /movies/yml/services.sh &')

#7 - Compile FFmpeg with GPU Support

In [ ]:
!curl -s https://raw.githubusercontent.com/theraw/gcolab-encoder/main/static/build_ffmpeg.sh > /tmp/ffmpeg.sh
!bash /tmp/ffmpeg.sh

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pkg-config is already the newest version (0.29.1-0ubuntu2).
libass-dev is already the newest version (1:0.14.0-1).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmp3lame-dev is already the newest version (3.100-2).
libopus-dev is already the newest version (1.1.2-1ubuntu1).
libx264-dev is already the newest version (2:0.152.2854+gite9a5903-2).
libx265-dev is already the newest version (2.6-3).
nasm is already the newest version (2.13.02-0.1).
libnuma-dev is already the newest version (2.0.11-2.1ubuntu0.1).
libvpx-dev is already the newest version (1.7.0-3ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'git' instead of 'git-core'
autocon